# Prototype of BIS2 project 

In [7]:
import geopandas
import pandas
import geojson
import requests
import json
from shapely.geometry import Point
import geopandas as gpd 
from shapely.geometry import Polygon

## Input
### Here we take the input from group 4. Obviously in this semplified version the input is just a point with the coordinates of Milan.

In [8]:
lat_point_list = [50.854457, 52.518172, 50.072651, 48.853033, 50.854457]
lon_point_list = [4.377184, 13.407759, 14.435935, 2.349553, 4.377184]

polygon_geom = Polygon(zip(lon_point_list, lat_point_list))
polygon = gpd.GeoDataFrame(index=[0], crs='epsg:4326', geometry=[polygon_geom])

In [9]:
lat = 45.464664
lon = 9.188540
id = 10

In [10]:
point = Point(lat, lon)

## API request
### The API request is executed with open-meteo.com

In [11]:
BASE_URL = "https://api.open-meteo.com/v1/forecast?latitude=" + str(lat) + "&longitude=" + str(
            lon) + "&hourly=temperature_2m,relativehumidity_2m,dewpoint_2m,apparent_temperature," \
                   "precipitation_probability,precipitation,rain,showers,snowfall,snow_depth,weathercode," \
                   "pressure_msl,surface_pressure,cloudcover,cloudcover_low,cloudcover_mid,cloudcover_high," \
                   "visibility,evapotranspiration,et0_fao_evapotranspiration,vapor_pressure_deficit," \
                   "windspeed_10m,windspeed_80m,windspeed_120m,windspeed_180m,winddirection_10m," \
                   "winddirection_80m,winddirection_120m,winddirection_180m,windgusts_10m,temperature_80m," \
                   "temperature_120m,temperature_180m,soil_temperature_0cm,soil_temperature_6cm," \
                   "soil_temperature_18cm,soil_temperature_54cm,soil_moisture_0_1cm,soil_moisture_1_3cm," \
                   "soil_moisture_3_9cm,soil_moisture_9_27cm,soil_moisture_27_81cm,is_day"
response = requests.get(BASE_URL)

response = response.json()

print(json.dumps(response, indent=4))

{
    "latitude": 45.46,
    "longitude": 9.18,
    "generationtime_ms": 15.362024307250977,
    "utc_offset_seconds": 0,
    "timezone": "GMT",
    "timezone_abbreviation": "GMT",
    "elevation": 142.0,
    "hourly_units": {
        "time": "iso8601",
        "temperature_2m": "\u00b0C",
        "relativehumidity_2m": "%",
        "dewpoint_2m": "\u00b0C",
        "apparent_temperature": "\u00b0C",
        "precipitation_probability": "%",
        "precipitation": "mm",
        "rain": "mm",
        "showers": "mm",
        "snowfall": "cm",
        "snow_depth": "m",
        "weathercode": "wmo code",
        "pressure_msl": "hPa",
        "surface_pressure": "hPa",
        "cloudcover": "%",
        "cloudcover_low": "%",
        "cloudcover_mid": "%",
        "cloudcover_high": "%",
        "visibility": "m",
        "evapotranspiration": "mm",
        "et0_fao_evapotranspiration": "mm",
        "vapor_pressure_deficit": "kPa",
        "windspeed_10m": "km/h",
        "windspeed_8

## Parsing of metereological data
### In this section the GeoPandas geoDataframe is built

In [12]:
gdf = geopandas.GeoDataFrame()

gdf['time'] = pandas.to_datetime(response['hourly']['time']).strftime('%Y-%m-%dT%H:%M:%S')
gdf['AOI_ID'] = id
gdf['EventID'] = id

gdf['temperature'] = response['hourly']['temperature_2m']
gdf['precipitationProbability'] = response['hourly']['precipitation_probability']
gdf['cloudcover'] =  response['hourly']['cloudcover']
gdf['day/night'] = response['hourly']['is_day']
gdf['geometry'] = polygon_geom
gdf['centroid'] = polygon_geom.centroid

gdf

,time,AOI_ID,EventID,temperature,precipitationProbability,cloudcover,day/night,geometry,centroid
0,2023-05-16T00:00:00,10,10,15.0,90,51,0,"POLYGON ((4.37718 50.85446, 13.40776 52.51817,...",POINT (8.981408476975249 50.56546094509109)
1,2023-05-16T01:00:00,10,10,14.9,79,81,0,"POLYGON ((4.37718 50.85446, 13.40776 52.51817,...",POINT (8.981408476975249 50.56546094509109)
2,2023-05-16T02:00:00,10,10,14.4,69,100,0,"POLYGON ((4.37718 50.85446, 13.40776 52.51817,...",POINT (8.981408476975249 50.56546094509109)
3,2023-05-16T03:00:00,10,10,14.1,58,100,0,"POLYGON ((4.37718 50.85446, 13.40776 52.51817,...",POINT (8.981408476975249 50.56546094509109)
4,2023-05-16T04:00:00,10,10,13.8,39,100,1,"POLYGON ((4.37718 50.85446, 13.40776 52.51817,...",POINT (8.981408476975249 50.56546094509109)
...,...,...,...,...,...,...,...,...,...
163,2023-05-22T19:00:00,10,10,22.0,25,40,0,"POLYGON ((4.37718 50.85446, 13.40776 52.51817,...",POINT (8.981408476975249 50.56546094509109)
164,2023-05-22T20:00:00,10,10,20.9,27,23,0,"POLYGON ((4.37718 50.85446, 13.40776 52.51817,...",POINT (8.981408476975249 50.56546094509109)
165,2023-05-22T21:00:00,10,10,19.5,29,6,0,"POLYGON ((4.37718 50.85446, 13.40776 52.51817,...",POINT (8.981408476975249 50.56546094509109)
166,2023-05-22T22:00:00,10,10,18.6,29,4,0,"POLYGON ((4.37718 50.85446, 13.40776 52.51817,...",POINT (8.981408476975249 50.56546094509109)


## Export
### At this point the ouput is created

In [13]:

# print(json_gdf)
# print(json.dumps(json_gdf))
# gdf['geometry'] = polygon_geom.__geo_interface__
gdf['centroid'] = json.dumps(point.__geo_interface__)

json_gdf = gdf.to_json()

print(gdf['centroid'])

x = json.loads(json_gdf)
print(json.dumps(x, indent=4))

# with open('my_file.json', 'w') as f:
#     f.write(json_gdf)


    


0      {"type": "Point", "coordinates": [45.464664, 9...
1      {"type": "Point", "coordinates": [45.464664, 9...
2      {"type": "Point", "coordinates": [45.464664, 9...
3      {"type": "Point", "coordinates": [45.464664, 9...
4      {"type": "Point", "coordinates": [45.464664, 9...
                             ...                        
163    {"type": "Point", "coordinates": [45.464664, 9...
164    {"type": "Point", "coordinates": [45.464664, 9...
165    {"type": "Point", "coordinates": [45.464664, 9...
166    {"type": "Point", "coordinates": [45.464664, 9...
167    {"type": "Point", "coordinates": [45.464664, 9...
Name: centroid, Length: 168, dtype: object
{
    "type": "FeatureCollection",
    "features": [
        {
            "id": "0",
            "type": "Feature",
            "properties": {
                "time": "2023-05-16T00:00:00",
                "AOI_ID": 10,
                "EventID": 10,
                "temperature": 15.0,
                "precipitationProbabilit

In [14]:
y = geojson.loads(json_gdf)
print(json.dumps(y, indent=4))

{
    "type": "FeatureCollection",
    "features": [
        {
            "type": "Feature",
            "id": "0",
            "geometry": {
                "type": "Polygon",
                "coordinates": [
                    [
                        [
                            4.377184,
                            50.854457
                        ],
                        [
                            13.407759,
                            52.518172
                        ],
                        [
                            14.435935,
                            50.072651
                        ],
                        [
                            2.349553,
                            48.853033
                        ],
                        [
                            4.377184,
                            50.854457
                        ]
                    ]
                ]
            },
            "properties": {
                "time": "2023-05-16T00